In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame(pd.read_csv('/Users/ycn/Desktop/Education/DM-HW1/retail.dat',sep='\t',header=None).apply(lambda row: set([int(item) for item in row[0].split(' ')[:-1]]),axis=1))
df

,0
0,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,"{32, 30, 31}"
2,"{33, 34, 35}"
3,"{36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46}"
4,"{48, 47, 38, 39}"
...,...
88157,"{14406, 39, 2665, 875, 2962, 14070, 16379, 155..."
88158,"{3591, 393, 522, 3722, 2830, 2449, 413, 799, 3..."
88159,"{4267, 2310}"
88160,"{48, 2528, 39}"


In [3]:
frequent = []
counter = 0
bad_subsets = []
min_sup = 4408
distinct_items = set()
def union_all(basket):
    global distinct_items
    distinct_items = distinct_items.union(basket)
    return basket
df[0].apply(union_all)
item_sets = [set([item]) for item in distinct_items]


def apriori(item_sets,distinct_items):
    global counter
    new_item_sets = []
    for item_set in item_sets:
        sup = len(df[df[0].apply(lambda x: item_set.issubset(x))])
        if sup >= min_sup:
            new_item_sets.append(item_set)
            frequent.append(item_set)
        else:
            bad_subsets.append(item_set)
            if not counter:
                distinct_items = distinct_items.difference(item_set)
    if item_sets == new_item_sets:
        return
    item_sets = new_item_sets
    new_item_sets = []
    for item_set in item_sets:
        for item in distinct_items:
            new_item_set = item_set.union(set([item]))
            bad_subset_flag = False
            for bad_subset in bad_subsets:
                if bad_subset.issubset(new_item_set):
                    bad_subset_flag = True
            if not item in item_set and not new_item_set in new_item_sets and not bad_subset_flag:
                new_item_sets.append(new_item_set)
    counter+=1
    apriori(new_item_sets,distinct_items)

apriori(item_sets,distinct_items)

In [24]:
temp = []
support_rules = {}
for item_set in frequent:
    support_rules[tuple(item_set)] = round(len(df[df[0].apply(lambda x: set(item_set).issubset(x))])/len(df),3)
from itertools import permutations
conf_rules = {}
lift_rules = {}
for item_set in frequent:
    if len(item_set) - 1:
        for perm in permutations(item_set):
            for k in range(1,len(item_set)):
                s1,s2 = set(perm[:k]),set(perm[k:])
                if not [s1,s2] in temp:
                    temp.append([s1,s2])
                    conf = len(df[df[0].apply(lambda x: s1.union(s2).issubset(x))])/len(df[df[0].apply(lambda x: s2.issubset(x))])
                    if conf >= 0.6: conf_rules[(tuple(s2),tuple(s1))] = round(conf,3)
                    lift_rules[(tuple(s1),tuple(s2))] = round((len(df[df[0].apply(lambda x: s1.union(s2).issubset(x))]) * len(df))/
                                                              (len(df[df[0].apply(lambda x: s2.issubset(x))])*len(df[df[0].apply(lambda x: s1.issubset(x))])),3)
print('support rules enforcing 0.05 threshold:')
print(support_rules)
print('conf rules enforcing 0.6 threshold:')
print(conf_rules)
for i in conf_rules:
    print(f'{i[0]}' +' => '+f'{i[1]}'+':'+f'{conf_rules[i]}',end=', ')
print('\nlift:')
print(lift_rules)

support rules enforcing 0.05 threshold:
{(32,): 0.172, (38,): 0.177, (39,): 0.575, (41,): 0.17, (48,): 0.478, (65,): 0.051, (32, 39): 0.096, (32, 48): 0.091, (38, 39): 0.117, (48, 38): 0.09, (41, 39): 0.129, (48, 39): 0.331, (48, 41): 0.102, (32, 48, 39): 0.061, (48, 38, 39): 0.069, (48, 41, 39): 0.084}
conf rules enforcing 0.6 threshold:
{((38,), (39,)): 0.663, ((41,), (39,)): 0.764, ((48,), (39,)): 0.692, ((41,), (48,)): 0.603, ((32, 39), (48,)): 0.639, ((32, 48), (39,)): 0.672, ((48, 38), (39,)): 0.768, ((41, 39), (48,)): 0.645, ((48, 41), (39,)): 0.817}
(38,) => (39,):0.663, (41,) => (39,):0.764, (48,) => (39,):0.692, (41,) => (48,):0.603, (32, 39) => (48,):0.639, (32, 48) => (39,):0.672, (48, 38) => (39,):0.768, (41, 39) => (48,):0.645, (48, 41) => (39,):0.817, 
lift:
{((32,), (39,)): 0.97, ((39,), (32,)): 0.97, ((32,), (48,)): 1.108, ((48,), (32,)): 1.108, ((38,), (39,)): 1.154, ((39,), (38,)): 1.154, ((48,), (38,)): 1.066, ((38,), (48,)): 1.066, ((41,), (39,)): 1.329, ((39,), (4